In [1]:
import numpy as np
import pandas as pd
import copy
import pickle

from util.util_load             import read_txt, read_scenario
from util.util_action           import evaluate_LocalCost

from VALIDATION.KeyRef_2.KeyRef_2_action_space import action_space


maxtime                = 10
PopSize                = 100
critical_machines      = {5, 6, 7, 8, 9, 10, 11, 12, 13, 21, 22, 26, 27}
action_name            = ["CDR1", "CDR2", "CDR3", "CDR4", "CDR5", "CDR6"]
K = 30

with open('VALIDATION/SMALL/pickle_JA_valid_scenarios_480.pkl', 'rb') as f:
    scenarios = pickle.load(f)

results    = []
ActionList = [0, 1, 2, 3, 4, 5]
InstanceList = [f'valid{i+1}' for i in range(15)]
ScenarioList = ['A', 'B', 'C']

for time_run in range(9):
    for action_id in ActionList:
        method = action_name[action_id]
        print(method)
        for instance_id in InstanceList:
            print(instance_id)
            for scenario_id in ScenarioList:
                data_path               = f"VALIDATION/SMALL/Case{instance_id}_480.txt"
                J, I, K, p_ijk, h_ijk,\
                d_j, n_j, MC_ji, n_MC_ji,  \
                OperationPool      = read_txt(data_path)
                
                JA_event =scenarios[instance_id + scenario_id]
                t                  = 0
                S_j                = np.zeros((J))
                S_k                = np.zeros((K))
                X_ijk              = np.zeros((I, J, K))
                S_ij               = np.zeros((I, J))
                C_ij               = np.zeros((I, J))
                JSet               = list(range(J))
                n_ops_left_j       = copy.deepcopy(n_j)
                T_cur              = np.zeros((J))


                # Random events
                while JSet:
                    action_method                   = action_space(J, K, p_ijk, h_ijk, d_j, n_j, 
                                                                    S_k, S_j, MC_ji, n_ops_left_j, 
                                                                    JSet, T_cur, X_ijk)					    
                    operation_machine_selection     = action_method[action_id]
                    i, j, k                         = operation_machine_selection()
                    X_ijk[i, j, k]             = 1
                    S_ij[i, j]                 = max(S_j[j], S_k[k])
                    C_ij[i, j]                 = S_ij[i, j] + p_ijk[i, j, k]  
                    S_k[k]                     = copy.deepcopy(C_ij[i, j])
                    

                    n_ops_left_j[j] -= 1
                    if n_ops_left_j[j] != 0:
                        S_j[j]   = copy.deepcopy(C_ij[i, j])
                        T_cur[j] = np.mean(S_k[h_ijk[i+1,j] == 1])
                        
                    else:
                        JSet.remove(j)
                        if j in JA_event:
                            J += 1
                            JSet.append(J-1)

                            # num operation of new job
                            n_newjob                    = copy.deepcopy(n_j[j])
                            n_j                         = np.append(n_j, n_newjob)
                            n_ops_left_j                = np.append(n_ops_left_j, n_newjob)
                            # processing time
                            p_newjob                    = copy.deepcopy(p_ijk[:, j, :])
                            p_newjob_reshape            = copy.deepcopy(p_newjob[:, np.newaxis, :])
                            p_ijk                       = np.concatenate((p_ijk, p_newjob_reshape), axis= 1)

                            # capable machine            
                            h_newjob                    = copy.deepcopy(h_ijk[:, j, :])
                            h_newjob_reshape            = copy.deepcopy(h_newjob[:, np.newaxis, :])
                            h_ijk                       = np.concatenate((h_ijk, h_newjob_reshape), axis= 1)

                            MC_newjob                   = copy.deepcopy(MC_ji[j])
                            MC_ji.append(MC_newjob)

                            X_ijk          		        = np.pad(X_ijk,((0, 0), (0, 1), (0, 0)), 	mode='constant', constant_values=0)
                            S_ij                        = np.pad(S_ij, ((0, 0), (0, 1)), 		 	mode='constant', constant_values=0)
                            C_ij                        = np.pad(C_ij, ((0, 0), (0, 1)),  			mode='constant', constant_values=0)

                            S_j                         = np.append(S_j, 0)

                            T_cur_newjob			    = np.mean(S_k[h_newjob[0] == 1])
                            T_cur                       = np.append(T_cur, T_cur_newjob)

                            deadline, description = JA_event[j]
                            if description == "urgent":
                                d_newjob = np.sum(np.sum(p_newjob*h_newjob, axis=1)/np.maximum(np.sum(h_newjob, axis=1),1)) *deadline
                            else:
                                d_newjob = deadline
                            d_j                   = np.append(d_j, d_newjob)

                tardiness, C_j = evaluate_LocalCost(d_j, C_ij, list(range(J)))
                results.append({
                        'TimeRun'   : time_run,
                        'Method'    : method,
                        'InstanceID': instance_id,
                        'ScenarioID': scenario_id,
                        'Tardiness' : tardiness
                        })

CDR1
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
valid10
valid11
valid12
valid13
valid14
valid15
CDR2
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
valid10
valid11
valid12
valid13
valid14
valid15
CDR3
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
valid10
valid11
valid12
valid13
valid14
valid15
CDR4
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
valid10
valid11
valid12
valid13
valid14
valid15
CDR5
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
valid10
valid11
valid12
valid13
valid14
valid15
CDR6
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
valid10
valid11
valid12
valid13
valid14
valid15
CDR1
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
valid10
valid11
valid12
valid13
valid14
valid15
CDR2
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
valid10
valid11
valid12
valid13
valid14
valid15
CDR3
valid1
valid2
valid3
valid4
valid5
valid6
valid7
valid8
valid9
vali

In [2]:
df = pd.DataFrame(results)
file_name = f"VALIDATION/KeyRef2_CDRs_add.xlsx"
df.to_excel(file_name, index=False)